In [1]:
import numpy as np
import numpy_indexed as npi
import itertools

In [16]:
#finding exact determinant as inbuilt functions approximate
def det(M):
    M = [row[:] for row in M] # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in range(N-1):
        if M[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if M[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in range(i+1,N):
            for k in range(i+1,N):
                assert ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) % prev == 0
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) // prev
        prev = M[i][i]
    return sign * M[-1][-1]

#Finds a sign vector for a vectorspace using the determinant method for finding circuits for a set of indices
def sign_vector(set_of_indices,basis_matrix,ambient_dimension):
    vector = np.zeros(ambient_dimension)
    submatrix = basis_matrix[:,set_of_indices]
    for i in range(ambient_dimension):
        if i in set_of_indices:
            vector[i] = 0
        else:
            vector[i] =det(np.column_stack((basis_matrix[:,i],submatrix)))
    return np.sign(vector)

#Same as sign_vector but returns the circuit instead of the sign vector
def circuit_vector(set_of_indices,basis_matrix,ambient_dimension):
    vector = np.zeros(ambient_dimension)
    submatrix = basis_matrix[:,set_of_indices]
    for i in range(ambient_dimension):
        if i in set_of_indices:
            vector[i] = 0
        else:
            vector[i] =det(np.column_stack((basis_matrix[:,i],submatrix)))
    return vector

#Finds all signvectors of a subspace by running sign_vector function over all index sets of the right size, and adding the negative of the signs found
def all_circuits(basis_matrix, ambient_dimension,subspace_dimension):
    result = []
    for subset in itertools.combinations(list(range(ambient_dimension)),subspace_dimension-1):
        result.append(sign_vector(list(subset),basis_matrix,ambient_dimension))
    result = np.array(result)
    # add the negative signs:
    result = np.vstack((result,-1*result))
    return npi.unique(np.astype(result,int))

#copy of all_circuits but using circuits instead of sign vectors and does not add negative copies, hence returns all circuits of a vectorspace
def test_all_circuits(basis_matrix, ambient_dimension,subspace_dimension):
    result = []
    for subset in itertools.combinations(list(range(ambient_dimension)),subspace_dimension-1):
        result.append(circuit_vector(list(subset),basis_matrix,ambient_dimension))
    result = np.array(result)
    return np.astype(result,int)

#Test if 2 vectors are conformal
def conformal_test(vector1,vector2):
    return not np.any(vector1*vector2<0)

#Tests if all vectors of an array of vectors are pairwise conformal
def is_subset_conformal(subset):
    #Check if all vectors in a subset are pairwise conformal.
    for i in range(len(subset)):
        for j in range(i + 1, len(subset)):
            if not conformal_test(subset[i], subset[j]):
                return False
    return True

#Takes aan array of conformal sign vectors and takes the union.
def union_sign_vectors(vectors):
    #Compute union of conformal sign vectors.
    result = np.zeros_like(vectors[0])
    for v in vectors:
        # Take the non-zero component if result is 0
        # If both are non-zero, they must be the same sign (already ensured)
        for i in range(len(v)):
            if result[i] == 0:
                result[i] = v[i]
    return result

#Takes an array of sign vectors and runs through all subsets, to test which are conformal. It then takes the conformal subsets and calculates the union, giving all possible sign vectors of from the signs inputted.
def add_union_of_conformal_sign_vectors(sign_vectors):
    #add the unions of sign vectors for all conformal subsets.
    total_sign_vectors = sign_vectors.copy()
    n = len(sign_vectors)
    for r in range(2, n + 1):
        test = len(total_sign_vectors)
        for subset in itertools.combinations(sign_vectors, r):
            if is_subset_conformal(subset):
                total_sign_vectors = np.vstack((total_sign_vectors,union_sign_vectors(subset)))
        total_sign_vectors = npi.unique(total_sign_vectors)
        if test == len(total_sign_vectors):
            break
    return total_sign_vectors


#Takes an array of sign vectors and removes any negative copies
def remove_opposite_rows(arr):
    result = []
    for row in arr:
        if not any(np.array_equal(row, -prev) for prev in result):
            result.append(row)
    return np.array(result)


#returns false if there is an entry of sign_vector which is different from the same entry in all of the sign vectors in the array set_of_signs
def test_if_vector_in_union(sign_vector,set_of_signs):
    for i in range(12):
        if np.all(np.array(set_of_signs)[:,i] != sign_vector[i]):
            return False
    return True

#  Removes negative copies of vectors from a list of lists. Keeps only one representative for each ±pair.
def remove_negative_copies(vectors):
    unique = []
    seen = set()

    for v in vectors:
        # Convert to tuple for hashability
        t = tuple(v)
        neg = tuple(-x for x in v)

        if neg not in seen and t not in seen:
            unique.append(v)
            seen.add(t)

    return unique

In [3]:
#This is the matrix A for which we wish to find the signs in the rowspace.
A = np.array([[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
              [0, 1, 1, 1, 2, 2, 3, 3, 0, 3, 2, 1],
              [0, 0, 0, -1, -1, -2, -2, -3, 1, -2, -1, 0]])
print(A)

[[ 1  0  1  1  1  1  1  1  0  1  1  1]
 [ 0  1  1  1  2  2  3  3  0  3  2  1]
 [ 0  0  0 -1 -1 -2 -2 -3  1 -2 -1  0]]


In [6]:
signA = add_union_of_conformal_sign_vectors(all_circuits(A,12,3))

In [4]:
test_all_circuits(A,12,3)

array([[ 0,  0,  0, -1, -1, -2, -2, -3,  1, -2, -1,  0],
       [ 0,  0,  0, -1, -1, -2, -2, -3,  1, -2, -1,  0],
       [ 0,  1,  1,  0,  1,  0,  1,  0,  1,  1,  1,  1],
       [ 0,  1,  1, -1,  0, -2, -1, -3,  2, -1,  0,  1],
       [ 0,  2,  2,  0,  2,  0,  2,  0,  2,  2,  2,  2],
       [ 0,  2,  2, -1,  1, -2,  0, -3,  3,  0,  1,  2],
       [ 0,  3,  3,  0,  3,  0,  3,  0,  3,  3,  3,  3],
       [ 0, -1, -1, -1, -2, -2, -3, -3,  0, -3, -2, -1],
       [ 0,  2,  2, -1,  1, -2,  0, -3,  3,  0,  1,  2],
       [ 0,  1,  1, -1,  0, -2, -1, -3,  2, -1,  0,  1],
       [ 0,  0,  0, -1, -1, -2, -2, -3,  1, -2, -1,  0],
       [ 0,  0,  0,  1,  1,  2,  2,  3, -1,  2,  1,  0],
       [-1,  0, -1,  0,  0,  1,  1,  2, -1,  1,  0, -1],
       [-1,  0, -1,  0,  0,  1,  1,  2, -1,  1,  0, -1],
       [-2,  0, -2, -1, -1,  0,  0,  1, -1,  0, -1, -2],
       [-2,  0, -2, -1, -1,  0,  0,  1, -1,  0, -1, -2],
       [-3,  0, -3, -2, -2, -1, -1,  0, -1, -1, -2, -3],
       [ 1,  0,  1,  1,  1,  1,

In [5]:
#this is a matrix with rows a basis for the image of the stoichiometric matrix in rowreduced form
S = np.array([[1,0,0,0,0,0,0,0,1,0,0,-1],
              [0,1,0,0,0,0,-1,0,-1,0,0,1],
              [0,0,1,0,0,0,-1,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,1,0,0,-1],
              [0,0,0,0,1,0,-1,0,0,0,0,0],
              [0,0,0,0,0,1,0,0,1,0,0,-1],
              [0,0,0,0,0,0,0,1,1,0,0,-1],
              [0,0,0,0,0,0,0,0,0,1,0,-1],
              [0,0,0,0,0,0,0,0,0,0,1,-1]])

In [8]:
#We now calculate positive and negative representations for all circuits in S
Scircuit_signs = all_circuits(S,12,9)

For each sign in A, we make a list of the signs in the Scircuit_signs that are conformal to that Sign. If a sign is in intersection it can be produced by conformal circuits of S. More precisly, we take a signvector a in signA and find all the signvectors s in Scircuit_signs such that Supp(s) \subseteq Supp(a) and the signs agree on the support.

In [9]:
Subsets = []
for chosen_vector in signA:
    result = []
    for vec in Scircuit_signs:
        if all(v == c or v == 0 for v, c in zip(vec, chosen_vector)):
            #Zip takes vec and chosen_vector and makes a list of 2-tuples with each being the pair of entries. This adds the additional property that all sign vectors in result have support which is a subset of the support of the chosen vector.
            result.append(vec)
    Subsets.append(result.copy())

For each a in signA we want to find a conformal subset of Scircuits_sign such that a is the union of that subset. We have already found the subset of Scircuiot_signs with all conformal signs so we just need to test if a is a union of a subset of the conformal vectors. Since all vectors we test against are conformal and have support included in the support of a, we only need to check  for each entry i that there is a signvector s in the subset with s[i]==a[i]. If a is the union of conformal signvectors in S we save the index of this a in a list.

In [12]:
intersection_indices = []
for i in range(len(Subsets)):
    if test_if_vector_in_union(signA[i],Subsets[i]):
        intersection_indices.append(i)

We then know that the sign vectors in sign(rowspace(A))\cap sign(Im(S)) are the sign vectors of signA that can be written as a sum of conformal sign vectors corresponding to circuits of S. i.e. in the follwing list.

In [13]:
Intersection = signA[intersection_indices]

This set includes negative copies, in the sense that if we have a we also -a so we remove these copies

In [24]:
pure_intersection = remove_opposite_rows(Intersection)


Now we save the vectors to a csv file. comented out so you don't accidentally save a bunch of copies

In [27]:
#np.savetxt("Clean_version_test_signs_in_intersection_n=3.csv", pure_intersection, delimiter=",")

We now want to find a vector in Im(S) that has the sign vector $\delta_2$:=[1,0,1,0,0,-1,-1,-1,1,-1,0,1]. We do this by calculating the circuits of S and summing circuits that are conformal to $\delta_2$.

In [46]:
Circuits_of_S = test_all_circuits(S,12,9)

In [47]:
np.sign([1,0,1,0,0,-1,-1,-1,1,-1,0,1])

array([ 1,  0,  1,  0,  0, -1, -1, -1,  1, -1,  0,  1])

In [48]:
Subset_of_circuits_in_S_conformal_to_delta_2 = []

result = []
for vec in Circuits_of_S:
    if all(v == c or v == 0 for v, c in zip(np.sign(vec),[1,0,1,0,0,-1,-1,-1,1,-1,0,1] )):
        #Zip takes vec and chosen_vector and makes a list of 2-tuples with each being the pair of entries. This adds the additional property that all sign vectors in result have support which is a subset of the support of the chosen vector.
        result.append(vec)
Subset_of_circuits_in_S_conformal_to_delta_2.append(result.copy())

In [49]:
print(len(Subset_of_circuits_in_S_conformal_to_delta_2[0]))

202


In [50]:
n = [0,0,0,0,0,0,0,0,0,0,0,0]
for i in Subset_of_circuits_in_S_conformal_to_delta_2[0]:
    n= n+i

In [51]:
print(n)

[ 19   0  12   0   0  -8 -12  -8   3 -18   0  15]


This vector does indeed have $\delta_2$ as its sign vector.